In [1]:
import pandas as pd
import numpy
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,r2_score
import yfinance as yf

#Used Models
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [2]:
symbol='^NSEI'
ticker = yf.Ticker(symbol)
data=ticker.history(period='max',interval='1m')
data.reset_index(inplace=True)

In [3]:
data.head()

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-04-15 09:43:00+05:30,22280.050781,22284.500000,22275.150391,22282.199219,0,0.0,0.0
1,2024-04-15 09:44:00+05:30,22281.699219,22290.199219,22278.349609,22280.699219,0,0.0,0.0
2,2024-04-15 09:45:00+05:30,22280.650391,22291.099609,22279.000000,22284.349609,0,0.0,0.0
3,2024-04-15 09:46:00+05:30,22285.050781,22289.849609,22280.900391,22288.500000,0,0.0,0.0
4,2024-04-15 09:47:00+05:30,22288.000000,22296.300781,22286.650391,22296.250000,0,0.0,0.0


In [4]:
data.tail()

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
1493,2024-04-22 09:37:00+05:30,22254.949219,22255.650391,22245.949219,22246.699219,0,0.0,0.0
1494,2024-04-22 09:38:00+05:30,22247.050781,22263.650391,22247.050781,22263.650391,0,0.0,0.0
1495,2024-04-22 09:39:00+05:30,22262.699219,22268.000000,22260.050781,22263.000000,0,0.0,0.0
1496,2024-04-22 09:40:00+05:30,22263.750000,22265.949219,22259.550781,22262.150391,0,0.0,0.0
1497,2024-04-22 09:41:00+05:30,22261.599609,22261.599609,22236.250000,22236.599609,0,0.0,0.0


In [5]:
# data = pd.read_csv('/content/Nifty50_shortTerm_5m_New.csv')

print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1498 entries, 0 to 1497
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype                       
---  ------        --------------  -----                       
 0   Datetime      1498 non-null   datetime64[ns, Asia/Kolkata]
 1   Open          1498 non-null   float64                     
 2   High          1498 non-null   float64                     
 3   Low           1498 non-null   float64                     
 4   Close         1498 non-null   float64                     
 5   Volume        1498 non-null   int64                       
 6   Dividends     1498 non-null   float64                     
 7   Stock Splits  1498 non-null   float64                     
dtypes: datetime64[ns, Asia/Kolkata](1), float64(6), int64(1)
memory usage: 93.8 KB
None


In [6]:
print(data.describe())


               Open          High           Low         Close  Volume  \
count   1498.000000   1498.000000   1498.000000   1498.000000  1498.0   
mean   22170.973286  22175.911833  22165.614734  22170.948323     0.0   
std      150.540115    149.697139    151.171745    150.343977     0.0   
min    21793.750000  21797.500000  21778.550781  21794.400391     0.0   
25%    22113.424805  22119.575195  22106.875000  22113.425293     0.0   
50%    22178.275391  22183.250000  22172.700195  22177.850586     0.0   
75%    22294.699219  22298.799805  22290.462402  22294.750000     0.0   
max    22425.949219  22426.550781  22420.000000  22424.550781     0.0   

       Dividends  Stock Splits  
count     1498.0        1498.0  
mean         0.0           0.0  
std          0.0           0.0  
min          0.0           0.0  
25%          0.0           0.0  
50%          0.0           0.0  
75%          0.0           0.0  
max          0.0           0.0  


In [7]:
data.drop_duplicates()

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2024-04-15 09:43:00+05:30,22280.050781,22284.500000,22275.150391,22282.199219,0,0.0,0.0
1,2024-04-15 09:44:00+05:30,22281.699219,22290.199219,22278.349609,22280.699219,0,0.0,0.0
2,2024-04-15 09:45:00+05:30,22280.650391,22291.099609,22279.000000,22284.349609,0,0.0,0.0
3,2024-04-15 09:46:00+05:30,22285.050781,22289.849609,22280.900391,22288.500000,0,0.0,0.0
4,2024-04-15 09:47:00+05:30,22288.000000,22296.300781,22286.650391,22296.250000,0,0.0,0.0
...,...,...,...,...,...,...,...,...
1493,2024-04-22 09:37:00+05:30,22254.949219,22255.650391,22245.949219,22246.699219,0,0.0,0.0
1494,2024-04-22 09:38:00+05:30,22247.050781,22263.650391,22247.050781,22263.650391,0,0.0,0.0
1495,2024-04-22 09:39:00+05:30,22262.699219,22268.000000,22260.050781,22263.000000,0,0.0,0.0
1496,2024-04-22 09:40:00+05:30,22263.750000,22265.949219,22259.550781,22262.150391,0,0.0,0.0


In [8]:
data.isnull().sum()

Datetime        0
Open            0
High            0
Low             0
Close           0
Volume          0
Dividends       0
Stock Splits    0
dtype: int64

In [9]:
data.drop(['Datetime','Volume','Dividends','Stock Splits'],axis=1,inplace=True)
data.head(10)



,Open,High,Low,Close
0,22280.050781,22284.500000,22275.150391,22282.199219
1,22281.699219,22290.199219,22278.349609,22280.699219
2,22280.650391,22291.099609,22279.000000,22284.349609
3,22285.050781,22289.849609,22280.900391,22288.500000
4,22288.000000,22296.300781,22286.650391,22296.250000
5,22295.650391,22296.699219,22287.599609,22295.050781
6,22294.699219,22295.550781,22283.349609,22294.099609
7,22293.650391,22300.900391,22291.250000,22299.750000
8,22300.250000,22301.599609,22290.699219,22292.400391
9,22292.650391,22300.050781,22291.500000,22295.699219


In [10]:
data['shift_close'] = data['Close'].shift(-1)
data.dropna(inplace=True)

In [11]:
data

,Open,High,Low,Close,shift_close
0,22280.050781,22284.500000,22275.150391,22282.199219,22280.699219
1,22281.699219,22290.199219,22278.349609,22280.699219,22284.349609
2,22280.650391,22291.099609,22279.000000,22284.349609,22288.500000
3,22285.050781,22289.849609,22280.900391,22288.500000,22296.250000
4,22288.000000,22296.300781,22286.650391,22296.250000,22295.050781
...,...,...,...,...,...
1492,22268.199219,22272.699219,22253.050781,22255.400391,22246.699219
1493,22254.949219,22255.650391,22245.949219,22246.699219,22263.650391
1494,22247.050781,22263.650391,22247.050781,22263.650391,22263.000000
1495,22262.699219,22268.000000,22260.050781,22263.000000,22262.150391


In [12]:
data.tail()

,Open,High,Low,Close,shift_close
1492,22268.199219,22272.699219,22253.050781,22255.400391,22246.699219
1493,22254.949219,22255.650391,22245.949219,22246.699219,22263.650391
1494,22247.050781,22263.650391,22247.050781,22263.650391,22263.000000
1495,22262.699219,22268.000000,22260.050781,22263.000000,22262.150391
1496,22263.750000,22265.949219,22259.550781,22262.150391,22236.599609


In [13]:
X=data.drop('shift_close',axis=1)
y = data['shift_close']

In [14]:
xy=pd.concat([X,y],axis=1)
xy.tail()

,Open,High,Low,Close,shift_close
1492,22268.199219,22272.699219,22253.050781,22255.400391,22246.699219
1493,22254.949219,22255.650391,22245.949219,22246.699219,22263.650391
1494,22247.050781,22263.650391,22247.050781,22263.650391,22263.000000
1495,22262.699219,22268.000000,22260.050781,22263.000000,22262.150391
1496,22263.750000,22265.949219,22259.550781,22262.150391,22236.599609


In [15]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [16]:

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

predictions = my_model.predict(X_test)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_test)))

print("R2 Score" + str(r2_score(y_test,predictions)))


print("Accuracy Score : "+str(my_model.score(X_test,y_test)))

predictions


Mean Absolute Error: 8.0268359375
R2 Score0.9864230332642533
Accuracy Score : 0.9864230332642533


array([21843.215, 22161.195, 22263.322, 22367.926, 22169.682, 21884.986,
       22195.236, 22192.148, 21866.896, 22191.846, 22036.396, 22297.033,
       22089.066, 22020.602, 22158.691, 22126.496, 22123.652, 22105.578,
       22048.68 , 22305.045, 22101.953, 21867.375, 21878.678, 22168.502,
       22331.67 , 22033.682, 22194.727, 22331.67 , 22144.664, 22131.81 ,
       22331.371, 22304.355, 22118.326, 22109.326, 22047.473, 21881.654,
       22398.293, 22375.676, 22335.64 , 22244.66 , 21945.46 , 22165.795,
       21925.92 , 22340.24 , 22401.482, 22129.021, 22202.537, 22217.93 ,
       22199.35 , 22001.248, 22370.22 , 22184.21 , 22203.543, 22087.734,
       22005.613, 22169.578, 22397.754, 22342.076, 21883.508, 22350.059,
       22384.697, 22364.377, 22199.059, 22024.527, 22193.854, 22131.71 ,
       22153.22 , 22140.256, 22281.123, 22339.69 , 21905.736, 22186.434,
       22010.766, 22254.178, 22242.086, 22177.225, 22199.059, 21881.576,
       21875.123, 21837.043, 22299.79 , 22137.326, 

In [17]:
RandomeForestModel = RandomForestRegressor()
RandomeForestModel.fit(X_train, y_train)


predictions = RandomeForestModel.predict(X_test)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_test)))

print("R2 Score" + str(r2_score(y_test,predictions)))


print("Accuracy Score : "+str(RandomeForestModel.score(X_test,y_test)))

predictions

Mean Absolute Error: 7.265521809895727
R2 Score0.9880373968266675
Accuracy Score : 0.9880373968266675


array([21847.92318359, 22158.5809375 , 22264.79134766, 22369.47783203,
       22169.21662109, 21885.02378906, 22195.26705078, 22191.80152344,
       21867.11458984, 22182.39982422, 22042.94431641, 22295.78400391,
       22075.90269531, 22020.72507813, 22159.37597656, 22123.98828125,
       22125.45527344, 22117.22560547, 22046.96433594, 22294.18765625,
       22101.57769531, 21867.165625  , 21891.68640625, 22167.88974609,
       22327.97759766, 22039.15929688, 22195.11775391, 22331.90328125,
       22145.09765625, 22133.36738281, 22336.94160156, 22305.66964844,
       22127.61326172, 22101.12306641, 22048.06388672, 21879.53396484,
       22396.15605469, 22373.97439453, 22336.12576172, 22264.17302734,
       21945.06716797, 22170.06845703, 21926.84138672, 22340.54023437,
       22409.17490234, 22125.48625   , 22200.44935547, 22206.50882812,
       22197.15130859, 22002.26128906, 22368.70054687, 22185.19015625,
       22200.64419922, 22091.47441406, 22014.59513672, 22170.02896484,
      

In [18]:
DesitionTreeModel = DecisionTreeRegressor()
DesitionTreeModel.fit(X_train, y_train)


predictions = DesitionTreeModel.predict(X_test)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_test)))

print("R2 Score" + str(r2_score(y_test,predictions)))


print("Accuracy Score : "+str(DesitionTreeModel.score(X_test,y_test)))

predictions

Mean Absolute Error: 9.250852864583333
R2 Score0.9839805963232485
Accuracy Score : 0.9839805963232485


array([21861.65039062, 22172.59960938, 22274.90039062, 22375.40039062,
       22169.90039062, 21867.75      , 22196.25      , 22188.65039062,
       21866.80078125, 22177.25      , 22043.09960938, 22296.84960938,
       22064.90039062, 22018.84960938, 22163.09960938, 22120.65039062,
       22120.        , 22121.19921875, 22054.69921875, 22283.        ,
       22087.80078125, 21872.75      , 21893.59960938, 22169.90039062,
       22326.65039062, 22049.34960938, 22196.84960938, 22321.75      ,
       22134.34960938, 22128.94921875, 22327.40039062, 22309.19921875,
       22133.34960938, 22107.44921875, 22044.75      , 21877.80078125,
       22399.19921875, 22374.94921875, 22334.30078125, 22255.25      ,
       21951.75      , 22180.19921875, 21928.75      , 22337.65039062,
       22413.15039062, 22133.75      , 22200.05078125, 22215.69921875,
       22194.40039062, 22001.34960938, 22368.84960938, 22189.30078125,
       22193.25      , 22091.55078125, 22017.65039062, 22178.30078125,
      

In [19]:

import pickle
pickle.dump(my_model,open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/XGBoost.pkl','wb'))
pickle.dump(RandomeForestModel,open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/RandomForestRegressor.pkl','wb'))
pickle.dump(DesitionTreeModel,open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/DesicionTreeModel.pkl','wb'))

# pickle.dump(my_model,open('/content/drive/MyDrive/AI COLLAGE/5m_Nifty_RandomForestRegressor.pkl','wb'))


import yfinance as yf
import datetime as dt
import pickle
from sklearn.metrics import mean_absolute_error,r2_score,accuracy


# Load the model from the pickle file
with open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/RandomForestRegressor.pkl', 'rb') as f:
    Rand = pickle.load(f)


with open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/DesicionTreeModel.pkl', 'rb') as f:
    Decision = pickle.load(f)


with open('/home/rahul/Desktop/Rahul/DE SEM 6/stock-prediction-bot/technical_analysis/technical_analysis_model/model_pkl_files/XGBoost.pkl', 'rb') as f:
    XgBoostModel = pickle.load(f)



while True:
    ticker = yf.Ticker('^NSEI')
    start=dt.datetime.now().strftime('%Y-%m-%d')
    end=(dt.datetime.now() + dt.timedelta(days=1)).strftime('%Y-%m-%d')

    data = ticker.history(interval='5m',start=start,end=end)
    data.reset_index(inplace=True)
    # print(data)
    data.columns = data.columns.str.strip()  # Remove extra spaces from column names

    data.drop(['Datetime', 'Volume', 'Dividends', 'Stock Splits'], inplace=True, axis=1)
    y = data['Close'].copy()
    data.drop(['Close'], inplace=True, axis=1)

    print("Random Forest Regressor")
    print("---------------------------\n")
    Rand_predic=Rand.predict(data)
    last_pred=Rand_predic[-1]
    print(f'Last Prediction ----------> :{ last_pred}')
    print(f'mae : {mean_absolute_error(y,Rand_predic)}')
    # print(f"Rand Accuracy Score :{}")
    print(f'r2 : {r2_score(y,Rand_predic)}')
    print("---------------------------")
    print("\n\n\n")

    print("Desicion Tree Regressor")
    print("---------------------------\n")
    DecisionPred=Decision.predict(data)
    last_pred=DecisionPred[-1]
    mae=mean_absolute_error(y,DecisionPred)
    # last_pred=last_pred+mae
    print(f'Last Prediction ----------> :{ last_pred}')
    print(f'mae : {mae}')
    print(f'r2 : {r2_score(y,DecisionPred)}')
    print("---------------------------")
    print("\n\n\n")

    print("XgBoostModel ")
    print("---------------------------\n")
    XgBoostPred=XgBoostModel.predict(data)
    last_pred=XgBoostPred[-1]
    print(f'Last Prediction ----------> :{ last_pred}')
    print(f'mae : {mean_absolute_error(y,XgBoostPred)}')
    print(f'r2 : {r2_score(y,XgBoostPred)}')
    print("---------------------------")
    print("\n\n\n")
    # data